In [301]:
import numpy as np
import random 
from icecream import ic

In [302]:
def fitness(solution, dist):
    sol = np.asarray(solution, dtype=int)
    edge_costs = dist[sol[:-1], sol[1:]].sum()
    edge_costs += dist[sol[-1], 0]
    return float(edge_costs)

def population_fitness(population, dist):
    pop = np.asarray(population, dtype=int)
    edge_costs = dist[pop[:, :-1], pop[:, 1:]].sum(axis=1)
    last_edges = dist[pop[:, -1], 0]
    return edge_costs + last_edges

def random_solution(n_cities):
    sol = list(range(n_cities))
    random.shuffle(sol)
    return sol
def nearest_neighbor_solution(dist, start_city=0):
    """
    Costruisce una soluzione greedy con nearest neighbor.
    Veloce e dà un buon punto di partenza.
    """
    n = dist.shape[0]
    unvisited = set(range(n))
    current = start_city
    solution = [current]
    unvisited.remove(current)
    
    while unvisited:
        # Trova la città non visitata più vicina
        nearest = min(unvisited, key=lambda city: dist[current, city])
        solution.append(nearest)
        unvisited.remove(nearest)
        current = nearest
    
    return solution
def init_population(pop_size, n_cities,dist):
    population = []
    
    # Numero di soluzioni greedy (max 10% della popolazione)
    n_greedy = max(1, min(pop_size // 10, n_cities // 2))
    
    # Crea soluzioni greedy partendo da città diverse
    greedy_starts = np.random.choice(n_cities, size=min(n_greedy, n_cities), replace=False)
    for start in greedy_starts:
        population.append(nearest_neighbor_solution(dist, start))  # placeholder
    
    # Riempie il resto con soluzioni random
    while len(population) < pop_size:
        population.append(random_solution(n_cities))
    
    return population

def tournament_selection(population, fitnesses, k):
    pop_size = len(population)
    idxs = np.random.randint(0, pop_size, size=k)
    best_idx = idxs[np.argmin(fitnesses[idxs])]
    return population[best_idx]


In [303]:
# swap two cities
def swap_mutation(sol):
    sol = np.asarray(sol, dtype=int).copy()
    i, j = np.random.randint(0, len(sol), size=2)
    sol[i], sol[j] = sol[j], sol[i]
    return sol
# reverse the order of cities between two indices
def inversion_mutation(sol):
    sol = np.asarray(sol, dtype=int).copy()
    i, j = np.random.randint(0, len(sol), size=2)
    if i > j:
        i, j = j, i
    sol[i:j+1] = sol[i:j+1][::-1]
    return sol
# scramble the order of cities between two indices
def scramble_mutation(sol):
    sol = np.asarray(sol, dtype=int).copy()
    i, j = np.random.randint(0, len(sol), size=2)
    if i > j:
        i, j = j, i
    segment = sol[i:j+1].copy()
    np.random.shuffle(segment)
    sol[i:j+1] = segment
    return sol
# remove a segment and insert it at a random position
def displacement_mutation(sol):
    sol = np.asarray(sol, dtype=int).copy()
    n = len(sol)
    if n < 4:
        return sol
    i, j = sorted(np.random.randint(0, n, size=2))
    if j - i < 1:
        return sol
    segment = sol[i:j+1].copy()
    remaining = np.concatenate([sol[:i], sol[j+1:]])
    insert_pos = np.random.randint(0, len(remaining) + 1)
    sol = np.concatenate([remaining[:insert_pos], segment, remaining[insert_pos:]])
    return sol

In [304]:
# Ordered Crossover (OX) 
def crossover_ox(p1, p2):
    parent1 = np.asarray(p1, dtype=int)
    parent2 = np.asarray(p2, dtype=int)
    size = len(parent1)
    i, j = np.random.randint(0, size, size=2)
    if i > j:
        i, j = j, i
    child = np.full(size, -1, dtype=int)
    child[i:j+1] = parent1[i:j+1]
    used = np.zeros(size, dtype=bool)
    used[child[i:j+1]] = True
    pos = (j + 1) % size
    for gene in np.concatenate((parent2[j+1:], parent2[:j+1])):
        if not used[gene]:
            child[pos] = gene
            used[gene] = True
            pos = (pos + 1) % size
    return child
# Partially Mapped Crossover (PMX) 
def crossover_pmx(p1, p2):
    """Partially Mapped Crossover (PMX) - migliore per ASIMMETRICI (preserva posizioni)"""
    parent1 = np.asarray(p1, dtype=int)
    parent2 = np.asarray(p2, dtype=int)
    size = len(parent1)
    i, j = np.random.randint(0, size, size=2)
    if i > j:
        i, j = j, i
    child = parent2.copy()
    segment = parent1[i:j+1]
    for idx in range(i, j+1):
        gene_from_p1 = parent1[idx]
        gene_from_p2 = parent2[idx]
        if gene_from_p1 != gene_from_p2:
            pos = np.where(child == gene_from_p1)[0][0]
            child[pos], child[idx] = child[idx], child[pos]
    return child

In [305]:
# local optimization move where you pick two edges in the tour, remove them, 
# reconnect the path by reversing the cities between them, and keep the change only if it makes the total route shorter
def two_opt(solution, dist, max_no_improve=50):
    sol = np.asarray(solution, dtype=int).copy()
    n = len(sol)
    best_cost = fitness(sol, dist)
    no_improve_count = 0
    for i in range(n - 1):
        if no_improve_count >= max_no_improve:
            break
        improved_in_i = False
        for j in range(i + 2, n):
            a, b = sol[i], sol[i+1]
            c, d = sol[j], sol[(j+1) % n]
            current_cost = dist[a, b] + dist[c, d]
            new_cost = dist[a, c] + dist[b, d]
            if new_cost < current_cost:
                sol[i+1:j+1] = sol[i+1:j+1][::-1]
                best_cost = best_cost - current_cost + new_cost
                improved_in_i = True
                no_improve_count = 0
                break
        if not improved_in_i:
            no_improve_count += 1
    return sol, best_cost

In [306]:
def is_symmetric(dist):
    return np.allclose(dist, dist.T)


In [307]:
# Ordered Crossover (OX)
# More probability of inversion mutation
# 2-opt at the end
# Bigger tournament size
def find_solution_symmetric(dist, generations=1000, mutation_rate=0.1, pop_size=500):
    n_cities = dist.shape[0]
    if n_cities <= 50:
        tournament_size = min(100, pop_size)
        apply_2opt = True
        max_2opt_improve = 100
    elif n_cities <= 200:
        tournament_size = min(80, pop_size)
        apply_2opt = True
        max_2opt_improve = 80
    else:
        tournament_size = min(60, pop_size)
        apply_2opt = True
        max_2opt_improve = 50
        pop_size = min(800, max(500, n_cities))
    population = init_population(pop_size, n_cities,dist)
    best_solution = None
    best_fitness = np.inf
    no_improvement = 0
    current_mutation_rate = mutation_rate
    for gen in range(generations):
        fitnesses = population_fitness(population, dist)
        gen_best_idx = np.argmin(fitnesses)
        gen_best = population[gen_best_idx]
        gen_best_fit = fitnesses[gen_best_idx]
        if best_solution is None or gen_best_fit < best_fitness:
            best_solution = gen_best.copy()
            best_fitness = gen_best_fit
            no_improvement = 0
            if current_mutation_rate > 0.05:
                current_mutation_rate *= 0.95
        else:
            no_improvement += 1
            if no_improvement % 15 == 0:
                current_mutation_rate = min(0.25, current_mutation_rate * 1.15)
        if no_improvement >= 50:
            break
        new_population = [best_solution]
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child = crossover_ox(parent1, parent2)
            if np.random.rand() < current_mutation_rate:
                if np.random.rand() < 0.8:
                    child = inversion_mutation(child)
                else:
                    child = swap_mutation(child)
            new_population.append(child)

        population = np.array(new_population, dtype=int)
    if apply_2opt:
        final_solution, final_fitness = two_opt(best_solution, dist, max_no_improve=max_2opt_improve)
        return final_solution
    
    return best_solution

In [308]:
# Partially Mapped Crossover (PMX)
# More diverse mutations
# Higher mutation rate
# No 2-opt
# Smaller tournament size
def find_solution_asymmetric(dist, generations=1000, mutation_rate=0.15, pop_size=500):
    n_cities = dist.shape[0]
    if n_cities <= 50:
        tournament_size = min(30, pop_size)
    elif n_cities <= 200:
        tournament_size = min(25, pop_size)
    else:
        tournament_size = min(20, pop_size)
        pop_size = min(1000, max(600, n_cities * 2))
    population = init_population(pop_size, n_cities,dist)
    best_solution = None
    best_fitness = np.inf
    no_improvement = 0
    current_mutation_rate = mutation_rate
    for gen in range(generations):
        fitnesses = population_fitness(population, dist)
        gen_best_idx = np.argmin(fitnesses)
        gen_best = population[gen_best_idx]
        gen_best_fit = fitnesses[gen_best_idx]
        if best_solution is None or gen_best_fit < best_fitness:
            best_solution = gen_best.copy()
            best_fitness = gen_best_fit
            no_improvement = 0
            if current_mutation_rate > 0.08:
                current_mutation_rate *= 0.96
        else:
            no_improvement += 1
            if no_improvement % 12 == 0:
                current_mutation_rate = min(0.35, current_mutation_rate * 1.2)

        if no_improvement >= 60: #more tolerance for asimmetrics
            break

        new_population = [best_solution]

        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child = crossover_pmx(parent1, parent2)

            if np.random.rand() < current_mutation_rate:
                rand = np.random.rand()
                if rand < 0.4:
                    child = swap_mutation(child)
                elif rand < 0.7:
                    child = scramble_mutation(child)
                else:
                    child = displacement_mutation(child)
            new_population.append(child)
        population = np.array(new_population, dtype=int)
    return best_solution

In [309]:
def find_solution_auto(dist, **kwargs):
    if is_symmetric(dist):
        print("SYMMETRIC PROBLEM")
        return find_solution_symmetric(dist, **kwargs)
    else:
        print("ASYMMETRIC PROBLEM")
        return find_solution_asymmetric(dist, **kwargs)

In [310]:
test=np.load("lab2/test_problem.npy")
print("test_problem.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test_problem.npy
Problema SIMMETRICO
Final best value: 2635.89


In [311]:
#g problems
test=np.load("lab2/problem_g_10.npy")
print("problem_g_10.npy")

solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)
test=np.load("lab2/problem_g_20.npy")
print("problem_g_20.npy")

solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_g_50.npy")
print("problem_g_50.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_g_100.npy")
print("problem_g_100.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_g_200.npy")
print("problem_g_200.npy")  
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_g_500.npy")
print("problem_g_500.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_g_1000.npy")
print("problem_g_1000.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)



problem_g_10.npy
Problema SIMMETRICO
Final best value: 1323.5724568998003
problem_g_20.npy
Problema SIMMETRICO
Final best value: 1644.3996571192388
problem_g_50.npy
Problema SIMMETRICO
Final best value: 3255.8474410100325
problem_g_100.npy
Problema SIMMETRICO
Final best value: 4766.187478516836
problem_g_200.npy
Problema SIMMETRICO
Final best value: 6167.15622212421
problem_g_500.npy
Problema SIMMETRICO
Final best value: 9905.490347225908
problem_g_1000.npy
Problema SIMMETRICO
Final best value: 13727.451339528996


In [312]:
test=np.load("lab2/problem_r1_10.npy")
print("problem_r1_10.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_20.npy")
print("problem_r1_20.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_50.npy")
print("problem_r1_50.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_100.npy")
print("problem_r1_100.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_200.npy")
print("problem_r1_200.npy")  
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_500.npy")
print("problem_r1_500.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r1_1000.npy")
print("problem_r1_1000.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)




problem_r1_10.npy
Problema ASIMMETRICO
Final best value: 155.2353290498573
problem_r1_20.npy
Problema ASIMMETRICO
Final best value: 329.66617692145905
problem_r1_50.npy
Problema ASIMMETRICO
Final best value: 595.2917750610328
problem_r1_100.npy
Problema ASIMMETRICO
Final best value: 720.7096301863269
problem_r1_200.npy
Problema ASIMMETRICO
Final best value: 1117.8449092671283
problem_r1_500.npy
Problema ASIMMETRICO
Final best value: 1738.956931025324
problem_r1_1000.npy
Problema ASIMMETRICO
Final best value: 2576.04015286527


In [313]:
test=np.load("lab2/problem_r2_10.npy")
print("problem_r2_10.npy")

solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)
test=np.load("lab2/problem_r2_20.npy")
print("problem_r2_20.npy")

solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r2_50.npy")
print("problem_r2_50.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r2_100.npy")
print("problem_r2_100.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r2_200.npy")
print("problem_r2_200.npy")  
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r2_500.npy")
print("problem_r2_500.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)

test=np.load("lab2/problem_r2_1000.npy")
print("problem_r1_1000.npy")
solution=find_solution_auto(test)
fitness_value=fitness(solution,test)
print("Final best value:",fitness_value)


problem_r2_10.npy
Problema ASIMMETRICO
Final best value: -411.7017155524984
problem_r2_20.npy
Problema ASIMMETRICO
Final best value: -842.0124445265324
problem_r2_50.npy
Problema ASIMMETRICO
Final best value: -2275.285403816136
problem_r2_100.npy
Problema ASIMMETRICO
Final best value: -4677.449826662115
problem_r2_200.npy
Problema ASIMMETRICO
Final best value: -9622.481826791609
problem_r2_500.npy
Problema ASIMMETRICO
Final best value: -24545.269595592807
problem_r1_1000.npy
Problema ASIMMETRICO
Final best value: -49443.7350731327
